<a href="https://colab.research.google.com/github/ramkumarr02/Titanic-Prediction-using-Pytorch/blob/master/Titanic_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Env Setup

## Packages



In [0]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

from sklearn.utils import shuffle
from torch.autograd import Variable

import warnings
warnings.filterwarnings("ignore")

## Mount Drive


In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Functions

## Split Columns

In [0]:
def column_split(df):

    try:
        df['SurName'], df['Name'] = df['Name'].str.split(',', 1).str
        df['Title'], df['Name'] = df['Name'].str.split('.', 1).str
        df['Cabin_Section'] = df[df['Cabin'].notna()]['Cabin'].astype(str).str[0]
        df['Cabin_Nums'] = df[df['Cabin'].notna()]['Cabin'].str.count(" ") + 1
        del df['Name']
        del df['Ticket']
        del df['Cabin']
        #del df['SurName']
        
    except:
        pass
    
    return(df)

## Impute Age by title

In [0]:
def impute_age_by_title(df):       
    train_with_age = df.copy()

    train_with_age_notnull = train_with_age[train_with_age['Age'].notna()]

    age_map = train_with_age_notnull.groupby(['Title'])['Age'].mean().astype('int').to_dict()
    age_map[' Ms'] = 28

    #temp = train_with_age['Age']

    for i, row in train_with_age.iterrows():           
        if pd.isnull(row['Age']):
            df.Age[i] = age_map[train_with_age.Title[i]]  

    return(df)

## Change all data type into int

In [0]:
def change_dtype(df):
    not_int_cols = list(df.select_dtypes(exclude=['int']).columns)
    df[not_int_cols] = df[not_int_cols].astype('int')
    return(df)

## Scale Data

In [0]:
def scale_data(df):

    scaled_features = StandardScaler().fit_transform(df.values)
    df = pd.DataFrame(scaled_features, index=df.index, columns=df.columns)
 
    return(df)

## PreProcess Data

In [0]:
def pre_process(df, prep_flag = None, train_data = None):        

    # Feature Engineering : Split Columns
    df = column_split(df)

    # Feature Engineering : Impute Age by Title
    df = impute_age_by_title(df)

    # One Hot Encoding
    df = pd.get_dummies(df)

    if prep_flag != None:
        train_data, df = train_data.align(df, join='left', axis=1)

    # Remove NaN
    df = df.fillna(0)

    # Change all Data types to Int
    df = change_dtype(df)

    # Scale Data
    scaled_df = scale_data(df)

    return(scaled_df)

## PD.DF to Tensor.DF

In [0]:
def df2df_tens(x,y):
    tens_x = torch.FloatTensor(x.values.astype('float'))
    tens_y = torch.LongTensor(y.values.astype('long'))
    
    df_tens = TensorDataset(tens_x, tens_y)    

    return(df_tens)

## Neural Net

In [0]:
class Net(nn.Module):
    def __init__(self, input_size, output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)        
        self.fc3 = nn.Linear(30, 2)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)                
        x = F.relu(x)
        x = self.fc3(x)                
        return(x)

## Model Trainer

In [0]:
def train_model(num_epochs, optimizer, criterion, train_loader, valid_loader, n_test):
    loss_list = []
    accuracy_list = []  

    for epoch in range(num_epochs):   
        for x,y in train_loader:        
            optimizer.zero_grad()
            yhat = model(x)
            loss =criterion(yhat, y)
            loss.backward()
            optimizer.step()

        correct = 0
        for x_test, y_test in valid_loader:        
            z = model(x_test)
            _, yhat = torch.max(z.data, 1)        
            correct += (yhat == y_test).sum().item()
        
        accuracy = correct/n_test        
        accuracy_list.append(accuracy)
        loss_list.append(loss.data)

        if epoch % 2 == 0:
            print(f'Epoch : {epoch}, Loss : {loss.data} , Accuracy : {accuracy}')        

    return(model, loss_list, accuracy_list)

# Code Engine

## Read Data

In [43]:
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Deep Learning/Titanic/PyTorch/train.csv')
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Deep Learning/Titanic/PyTorch/test.csv')

del train['PassengerId']
del test['PassengerId']

train_copy = train.copy()
test_copy = test.copy()

train.head(2)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


## Split Data for OOB testing

In [0]:
train = pd.concat([train]*20, ignore_index=True)

In [0]:
train_x, valid_x, train_y, valid_y = train_test_split(train.loc[:, train.columns != 'Survived'], train['Survived'],train_size = 0.8,random_state = 1)

## PreProcess Data

In [0]:
scaled_train_x = pre_process(train_x)
scaled_valid_x = pre_process(valid_x, train_data = scaled_train_x, prep_flag = 'valid')

In [0]:
# Convert Dataframe to Tensor Dataframe 
train_tensor = df2df_tens(scaled_train_x,train_y)
valid_tensor = df2df_tens(scaled_valid_x,valid_y)

In [0]:
# Data Loader : Train and valid
train_loader = DataLoader(dataset = train_tensor, batch_size=batch_size, shuffle = True)
valid_loader = DataLoader(dataset = valid_tensor, batch_size=batch_size, shuffle = True)

# Modelling

## Parameters

In [0]:
batch_size = 500

#input_size = 589
input_size = 703
output_size = 2
num_epochs = 10
learning_rate = 0.01
criterion = nn.CrossEntropyLoss()

n_test = len(scaled_valid_x)

In [0]:
model = Net(input_size = input_size, output_size = output_size)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## Training

In [52]:
model, loss_list, accuracy_list = train_model(num_epochs, optimizer, criterion, train_loader, valid_loader, n_test)

Epoch : 0, Loss : 0.02535015158355236 , Accuracy : 0.9910213243546577
Epoch : 2, Loss : 0.006903147324919701 , Accuracy : 0.9935465768799102
Epoch : 4, Loss : 0.008495423942804337 , Accuracy : 0.9941077441077442
Epoch : 6, Loss : 0.004000845365226269 , Accuracy : 0.9971941638608305
Epoch : 8, Loss : 0.002794662257656455 , Accuracy : 0.9980359147025814


# Testing

## Prep Test data

In [54]:
scaled_test_x = pre_process(test, train_data = scaled_train_x, prep_flag = 'Test')
test_x_tens = torch.FloatTensor(scaled_test_x.values.astype('float'))

z_test = model(test_x_tens)
_, yhat = torch.max(z_test.data, 1)
print(len(yhat))
yhat

418


tensor([0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0,
        1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,
        1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0,
        1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0,
        1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0,
        1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
        1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0,
        1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1,
        0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1,
        1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0,
        1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
        0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1,
        0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1,